In [1]:
import os
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from dateutil.parser import parse
import datetime
os.getcwd()

'/Linux/[2021, 2022] 초분광(Hyperspectral) 데이터/[ 2022 ]/PSC'

# **Dataset preprocessing**

In [2]:
def data_set (data) :
    day_df = pd.DataFrame(columns = ["Year", "Month", "Day", "Hour", "Start Min", "End Min"] + list(data.columns))

    for i in tqdm(range(data.shape[0])) : 
        obs = data.iloc[[i], :]                                         # obs : 데이터 하나
        time = parse(obs["timestamp"].values[0])                        # obs의 time 정보
        time = time + datetime.timedelta(hours = 9)

        # start = 0이면 해당 날짜의 데이터 수집 시작 알림
        if (time.hour == 0) & (time.minute < 10) :
            print("{}년 {}월 {}일 데이터 수집 시작".format(time.year, time.month, time.day))
        
        # obs의 관측 time(hour)이 하루가 지나면 데이터 수집 종료
        elif (time.hour == 23) & (time.minute > 50) :
            print("{}년 {}월 {}일 데이터 수집 종료".format(time.year, time.month, time.day))

        # obs의 관측 time(hour)이 0~23이면 데이터 수집
        if time.hour in range(0, 24) :
            Y, M, D, H, Min = time.year, time.month, time.day, time.hour, time.minute

            if time.minute in range(0, 15+1) :
                inform = pd.DataFrame([Y, M, D, H, 0, 15] + list(obs.values[0]), index = day_df.columns).T
                day_df = pd.concat([day_df, inform], axis = 0)

            elif time.minute in range(15, 30+1) :
                inform = pd.DataFrame([Y, M, D, H, 15, 30] + list(obs.values[0]), index = day_df.columns).T
                day_df = pd.concat([day_df, inform], axis = 0)

            elif time.minute in range(30, 45+1) :
                inform = pd.DataFrame([Y, M, D, H, 30, 45] + list(obs.values[0]), index = day_df.columns).T
                day_df = pd.concat([day_df, inform], axis = 0)

            elif time.minute in range(45, 60+1) :
                inform = pd.DataFrame([Y, M, D, H, 45, 0] + list(obs.values[0]), index = day_df.columns).T
                day_df = pd.concat([day_df, inform], axis = 0)
        

    day_df = day_df.drop(columns = ["timestamp"])
    complete_data = day_df.groupby(["Year", "Month", "Day", "Hour", "Start Min", "End Min"]).sum()
    
    return complete_data

In [3]:
def data_set2 (data) :
    day_df = pd.DataFrame(columns = ["Year", "Month", "Day", "Hour", "Min"] + list(data.columns))

    for i in tqdm(range(data.shape[0])) : 
        obs = data.iloc[[i], :]                                         # obs : 데이터 하나
        time = parse(obs["timestamp"].values[0])                        # obs의 time 정보
        time = time + datetime.timedelta(hours = 9)

        Y, M, D, H, Min = time.year, time.month, time.day, time.hour, time.minute
        
        if time.minute in range(0, 15) :
            inform = pd.DataFrame([Y, M, D, H, 15] + list(obs.values[0]), index = day_df.columns).T
            day_df = pd.concat([day_df, inform], axis = 0)

        elif time.minute in range(15, 30) :
            inform = pd.DataFrame([Y, M, D, H, 30] + list(obs.values[0]), index = day_df.columns).T
            day_df = pd.concat([day_df, inform], axis = 0)

        elif time.minute in range(30, 45) :
            inform = pd.DataFrame([Y, M, D, H, 45] + list(obs.values[0]), index = day_df.columns).T
            day_df = pd.concat([day_df, inform], axis = 0)

        elif time.minute in range(45, 60) :
            time2 = time + datetime.timedelta(minutes = 15)
            Y_, M_, D_, H_= time2.year, time2.month, time2.day, time2.hour
            inform = pd.DataFrame([Y_, M_, D_, H_, 0] + list(obs.values[0]), index = day_df.columns).T
            day_df = pd.concat([day_df, inform], axis = 0)


    day_df = day_df.drop(columns = ["timestamp"]).reset_index(drop = True)
        
    return day_df

In [4]:
data = pd.read_csv("../DAT/yangpyeong_dataset_null.csv")
print(data.shape)
data.head()

(31265, 565)


,airTemperature,chlorophyll,conductivity,do,humidity,insolation,ph,phycocyanin,reflectance.000,reflectance.001,...,reflectance.547,reflectance.548,reflectance.549,reflectance.550,timestamp,turbidity,waterDepth,waterTemperature,windDirection,windSpeed
0,NaN,3.3,0.26,9.92,NaN,NaN,8.226,0.42,NaN,NaN,...,NaN,NaN,NaN,NaN,20211101T001126,12.03,0.68,15.8,NaN,NaN
1,10.3,NaN,NaN,NaN,100.0,175.9,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,20211101T001128,NaN,NaN,NaN,10.4,0.9
2,NaN,3.3,0.26,9.92,NaN,NaN,8.226,0.42,NaN,NaN,...,NaN,NaN,NaN,NaN,20211101T002625,12.03,0.68,15.8,NaN,NaN
3,10.4,NaN,NaN,NaN,100.0,189.8,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,20211101T002627,NaN,NaN,NaN,23.8,1.4
4,NaN,3.3,0.26,9.92,NaN,NaN,8.226,0.42,NaN,NaN,...,NaN,NaN,NaN,NaN,20211101T004125,12.03,0.68,15.8,NaN,NaN


In [5]:
if "chla" in list(data.columns) :
    water = data[["timestamp", 'chla', 'chlorophyll', 'conductivity', 'do', 'ph', 'phycocyanin', 'turbidity', 'waterDepth', 'waterTemperature']].dropna().reset_index(drop = True)
else :
    water = data[["timestamp", 'chlorophyll', 'conductivity', 'do', 'ph', 'phycocyanin', 'turbidity', 'waterDepth', 'waterTemperature']].dropna().reset_index(drop = True)
weather = data[["timestamp", 'airTemperature', 'humidity', 'insolation', 'windDirection', 'windSpeed']].dropna().reset_index(drop = True)
hs = data[["timestamp"] + list(data.filter(regex = 'reflectance').columns)].dropna().reset_index(drop = True)

print(water.shape, weather.shape, hs.shape)

(14781, 9) (14941, 6) (1543, 552)


In [6]:
new_water = data_set2(water)
new_weather = data_set2(weather)
new_hs = data_set2(hs)

  0%|          | 0/14781 [00:00<?, ?it/s]

  0%|          | 0/14941 [00:00<?, ?it/s]

  0%|          | 0/1543 [00:00<?, ?it/s]

In [7]:
display(new_water.head(3))
display(new_weather.head(3))
display(new_hs.head(3))

,Year,Month,Day,Hour,Min,chlorophyll,conductivity,do,ph,phycocyanin,turbidity,waterDepth,waterTemperature
0,2021,11,1,9,15,3.3,0.26,9.92,8.226,0.42,12.03,0.68,15.8
1,2021,11,1,9,30,3.3,0.26,9.92,8.226,0.42,12.03,0.68,15.8
2,2021,11,1,9,45,3.3,0.26,9.92,8.226,0.42,12.03,0.68,15.8


,Year,Month,Day,Hour,Min,airTemperature,humidity,insolation,windDirection,windSpeed
0,2021,11,1,9,15,10.3,100,175.9,10.4,0.9
1,2021,11,1,9,30,10.4,100,189.8,23.8,1.4
2,2021,11,1,9,45,10.8,100,163.9,289.4,0.4


,Year,Month,Day,Hour,Min,reflectance.000,reflectance.001,reflectance.002,reflectance.003,reflectance.004,...,reflectance.541,reflectance.542,reflectance.543,reflectance.544,reflectance.545,reflectance.546,reflectance.547,reflectance.548,reflectance.549,reflectance.550
0,2021,11,1,10,15,0.00396732,0.00393777,0.00384069,0.0037979,0.00375127,...,0.00088871,0.00089527,0.00090657,0.0008776,0.00088078,0.00083906,0.00078705,0.00073717,0.00067999,0.00072323
1,2021,11,1,10,30,0.0039815,0.00394513,0.00387614,0.00382533,0.00379026,...,0.00039792,0.00039655,0.00036787,0.00033992,0.00031241,0.00028047,0.00024021,0.00023878,0.0002161,0.00022424
2,2021,11,1,10,45,0.0038206,0.00378978,0.00371596,0.00366487,0.00360389,...,0.00060992,0.00060086,0.00060226,0.00059648,0.00057783,0.00054693,0.00049905,0.00046669,0.00043932,0.00041765


In [8]:
new_data = pd.merge(left = new_water, right = new_weather, how = 'outer', on = ["Year", "Month", "Day", "Hour", "Min"], sort = True)
new_data = pd.merge(left = new_data, right = new_hs, how = 'outer', on = ["Year", "Month", "Day", "Hour", "Min"], sort = True)  
new_data = new_data.astype({"Year" : "int", "Month" : "int", "Day" : "int", "Hour" : "int", "Min" : "int"})
new_data.head(3)

,Year,Month,Day,Hour,Min,chlorophyll,conductivity,do,ph,phycocyanin,...,reflectance.541,reflectance.542,reflectance.543,reflectance.544,reflectance.545,reflectance.546,reflectance.547,reflectance.548,reflectance.549,reflectance.550
0,2021,11,1,9,15,3.3,0.26,9.92,8.226,0.42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021,11,1,9,30,3.3,0.26,9.92,8.226,0.42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021,11,1,9,45,3.3,0.26,9.92,8.226,0.42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# new_data.to_csv("../DAT/complete_data(yp).csv", index = False)